In [ ]:
import numpy
import os
import sklearn.ensemble
import sklearn.neighbors
import sklearn.neural_network
import sklearn.svm
import sklearn.tree

from samples import get_index

In [ ]:
cfg = {
    "fold": 5,
    "n_labels": 5,
    "path_base": "dataset",
    "path_out": "out",
    "pca": True,
    "test_size": 0.2,
    "train_size": 0.8,
}
list_index = get_index(cfg, os.path.join(cfg["path_base"], "surf64.txt"))

In [ ]:
for classifier in list(sklearn.tree.DecisionTreeClassifier(random_state=1), sklearn.neighbors.KNeighborsClassifier(n_jobs=-1), sklearn.neural_network.MLPClassifier(random_state=1), sklearn.ensemble.RandomForestClassifier(random_state=1), sklearn.svm.SVC(random_state=1, )):

